In [33]:
from tableone import TableOne
import pandas as pd
import os

data = pd.read_csv(f'/Users/Tristan/Documents/Projekte/Boston Celi/1 SaO2-SpO2/pulse-ox-dataset/data/pulseOx_dataset.csv')


/var/folders/78/7bh523553yl6dn_r17wfrql40000gn/T/ipykernel_35109/2601315501.py:5: DtypeWarning: Columns (0,30) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(f'/Users/Tristan/Documents/Projekte/Boston Celi/1 SaO2-SpO2/pulse-ox-dataset/data/pulseOx_dataset.csv')


Aggregate Data on the patient level first

In [42]:
# print unique values of sex_female for source_db = 'eicu'
# 
df = data.groupby('source_db').apply(lambda x: x['sex_female'].unique())
print(df)

source_db
eicu            [0]
mimic_iii    [0, 1]
mimic_iv     [0, 1]
dtype: object


In [36]:
# aggregation if we have more than one pair

# get anchor year as var?

In [39]:
# Groupby Variable
groupby = "source_db"
source = "source_db"

# Encode NA as 0, if missing means 0
cols_na = ['comorbidity_score_value', 'in_hospital_mortality']

for c in cols_na:
    data[c] = data[c].fillna(0)

# Continuous Variables
data['los_hosp_dead'] = data[data.in_hospital_mortality == 1].los_hospital
data['los_hosp_surv'] = data[data.in_hospital_mortality == 0].los_hospital

data['los_icu_dead'] = data[data.in_hospital_mortality == 1].los_ICU
data['los_icu_surv'] = data[data.in_hospital_mortality == 0].los_ICU

# Column methehemoglobin has ___, remove thhe underscore
#data.Methemoglobin = data.Methemoglobin.apply(lambda x: x.replace("___", ""))

# Convert "Methemoglobin" column to numeric and replace non-numeric values (including double underscores) with NaN
data['Methemoglobin'] = pd.to_numeric(data['Methemoglobin'], errors='coerce')


order = {
    "race_ethnicity": ["White", "Black", "Hispanic OR Latino", "Asian", 
                       "Unknown",  'American Indian / Alaska Native',
                        'Native Hawaiian / Pacific Islander', 'More Than One Race'],
    "sex_female": [1, 0],
    "in_hospital_mortality": [1, 0],
    "region": ["Midwest", "Northeast", "South", "West"],
    "numbedscategory": ['>= 500', '100 - 249', '<100', '250 - 499'],
    'teachingstatus': ['TRUE', 'FALSE'],
}

limit = {
    "sex_female": 1,
    "in_hospital_mortality": 1,
}

categ = ['sex_female',
        'in_hospital_mortality', 
        'race_ethnicity',
        "region",
        "numbedscategory",
        'teachingstatus',
    ]

nonnorm = ['admission_age', 
            'BMI_admission',
            'los_icu_dead',
            'los_icu_surv',
            'sofa_past_overall_24hr', # error need to get first 24 hours
            'delta_sofa_past_overall_24hr',
            'sofa_future_overall_24hr',
            'delta_sofa_future_overall_24hr',
            'comorbidity_score_value', # error either CCI or Elix
            'pH', 
            'pCO2', 
            'pO2',
            'SaO2',
            'SpO2',
            'Carboxyhemoglobin',
            'Methemoglobin',
            'vitals_heart_rate',
            'vitals_resp_rate',
            'vitals_mbp_i',
            'vitals_tempc',
        ]

labls = {
    'race_ethnicity': 'Race-ethnicity',
    'admission_age': 'Age',
    'sex_female': 'Sex female',
    'in_hospital_mortality': "In-Hospital Mortality",
    "region": 'US Census Region',
    "numbedscategory": 'No. of Beds',
    'teachingstatus': 'Teaching Hospital',
    'los_icu_dead': "ICU LOS (days, if deceased)",
    'los_icu_surv': "ICU LOS (days, if survived)",
    'comorbidity_score_value': "Charlson Comorbidity Index",
    'sofa_past_overall_24hr': "SOFA Score before pair",
    'delta_sofa_past_overall_24hr': "SOFA Time delta before pair",
    'sofa_future_overall_24hr': "SOFA Score after pair",
    'delta_sofa_future_overall_24hr': "SOFA Time delta after pair",    
    'vitals_heart_rate': "Heart Rate before pair",
    'vitals_resp_rate': "Respiratory Rate before pair",
    'vitals_mbp_i': "Mean Blood Pressure before pair",
    'vitals_tempc': "Temperature before pair",
    }

decimals = {
    'admission_age': 0,
    'BMI_admission': 1,
    'pH': 0,
    'los_icu_dead': 2,
    'los_icu_surv': 2, 
    }

# Create a TableOne 
table1 = TableOne(data, columns=categ+nonnorm,
                    rename=labls, limit=limit, order=order, decimals=decimals,
                    groupby=groupby, categorical=categ, nonnormal=nonnorm,
                    missing=False, overall=True,
                    dip_test=True, normal_test=True, tukey_test=True, htest_name=True)

# Write table 1 to excel
try:
    table1.to_excel(f"/Users/Tristan/Documents/Projekte/Boston Celi/1 SaO2-SpO2/pulse-ox-dataset/results/table1/groupby_{source}.xlsx")
# if folder does not exist, create it and save results
except:
    os.mkdir("/Users/Tristan/Documents/Projekte/Boston Celi/1 SaO2-SpO2/pulse-ox-dataset/results/table1")
    table1.to_excel(f"/Users/Tristan/Documents/Projekte/Boston Celi/1 SaO2-SpO2/pulse-ox-dataset/results/table1/groupby_{source}.xlsx")




/Users/Tristan/Library/Python/3.10/lib/python/site-packages/tableone/tableone.py:1441: UserWarning: Order value not found: in_hospital_mortality: ['1', '0']
  warnings.warn(("Order value not found: "
/Users/Tristan/Library/Python/3.10/lib/python/site-packages/tableone/tableone.py:1441: UserWarning: Order value not found: teachingstatus: ['TRUE', 'FALSE']
  warnings.warn(("Order value not found: "
